# Tile z and zo 
Tile the ASCII text files containing x, y, z and zo information.

In [2]:
import pdal
import json
import geopandas
import shapely
import shapely.wkt
import pathlib
import shutil 
import rioxarray
import xarray
import numpy
import rioxarray.merge

## Copy locally

In [3]:
crs = 32758
z_low_noise = -10

In [4]:
# Setup remote and local paths
remote_path = pathlib.Path(r"\\niwa.local\projects\hamilton\GNS23201\Working\Flooding\Roughness\Z_Zo_Tiles")
local_path = pathlib.Path(r"C:\Local\data\GNS23201\tiles")
(local_path / "raw").mkdir(exist_ok = True)
(local_path / "las").mkdir(exist_ok = True)
(local_path / "zo").mkdir(exist_ok = True)

# Z processing
## Convert to LAS

In [5]:
# Convert each to las
for file in (local_path / "raw").iterdir():
    #print(f"Converting {file.name} to LAS")
    pdal_pipeline_instructions = [
        {
            "type": "readers.text",
            "filename": str(file),
            "header": "X, Y, Z, Intensity"
        },
        {
            "type":"writers.las",
            "filename":str(file.parent.parent / "las" / f"{file.stem}.las"),
            "a_srs": f"EPSG:{crs}"
        }
    ]
    pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
    pdal_pipeline.execute()

## Combine into a single LAZ

In [6]:
# Create instruction
pdal_pipeline_instructions = []
for file in (local_path / "las").iterdir():
    if file.suffix == '.las':
        pdal_pipeline_instructions.append(
            {"type": "readers.las",
            "filename": str(file),})
pdal_pipeline_instructions.append({"type": "filters.merge",})
pdal_pipeline_instructions.append(
    {"type":"writers.las",
     "filename": str(local_path / "las" / "combined.laz"),
     "a_srs": f"EPSG:{crs}",
     "compression": "laszip"})
# Run pipeline
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

## Remove noise points

In [ ]:
pdal_pipeline_instructions = [ 
    { "type": "readers.las", "filename": str(local_path / "las" / "combined.laz"), }
]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();
# Remove noise
points = pdal_pipeline.arrays[0]
points = points[points['Z'] > z_low_noise]
# Write out value
pdal_pipeline_instructions = [
            {"type": "writers.las", "filename": str(local_path / "las" / "combined_no_noise.laz"),
             "compression": "laszip",
             "a_srs": f"EPSG:{crs}"
            }
        ]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [points])
pdal_pipeline.execute()
# Write out crs information
dem = rioxarray.open_rasterio(local_path / "las" / "combined.tif")
dem = dem.rio.write_crs(crs)
dem.rio.to_raster(local_path / "las" / "combined.tif")

In [ ]:
pdal_pipeline_instructions = [
            {"type": "writers.las", "filename": str(local_path / "las" / "combined_no_noise.laz"),
                "compression": "laszip",
            }
        ]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [points])
pdal_pipeline.execute()

## Rasterise at 1m and save as a Geotiff

In [ ]:
pdal_pipeline_instructions = [ 
    { "type": "readers.las", "filename": str(local_path / "las" / "combined_no_noise.laz"), },
    { "type": "filters.delaunay" },
    { "type": "filters.faceraster", "resolution": 1 },
    { "type": "writers.raster", "filename": str(local_path / "las" / "combined.tif") }
]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

# zo processing
## Save a LAZ with zo as Z

In [ ]:
# Convert each to las
for file in (local_path / "raw").iterdir():
    #print(f"Converting {file.name} to LAS")
    pdal_pipeline_instructions = [
        {
            "type": "readers.text",
            "filename": str(file),
            "header": "X, Y, zo, Z"
        },
        {
            "type":"writers.las",
            "filename":str(file.parent.parent / "zo" / f"{file.stem}.las"),
            "a_srs": f"EPSG:{crs}"
        }
    ]
    pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
    pdal_pipeline.execute()

## Combine into a single LAZ

In [ ]:
# Create instruction
pdal_pipeline_instructions = []
for file in (local_path / "zo").iterdir():
    if file.suffix == '.las':
        pdal_pipeline_instructions.append(
            {"type": "readers.las",
            "filename": str(file),})
pdal_pipeline_instructions.append({"type": "filters.merge",})
pdal_pipeline_instructions.append(
    {"type":"writers.las",
     "filename": str(local_path / "zo" / "combined.laz"),
     "compression": "laszip"})
# Run pipeline
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

## Rasterise 1m and save as a Geotiff

## Get catchment outline

In [ ]:
pdal_pipeline_instructions = [ 
    { "type": "readers.las", "filename": str(local_path / "zo" / "combined.laz"), },
    { "type": "filters.delaunay" },
    { "type": "filters.faceraster", "resolution": 1 },
    { "type": "writers.raster", "filename": str(local_path / "zo" / "combined.tif") }
]
pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();
# Write out crs information
dem = rioxarray.open_rasterio(local_path / "zo" / "combined.tif")
dem = dem.rio.write_crs(crs)
dem.rio.to_raster(local_path / "zo" / "combined.tif")

# Remove outliars, re-interpolate and clip
## Get boundary

## Read in Geotiffs and clip

In [ ]:
polygon = geopandas.read_file(local_path / "combined_boundary_manual_crs.geojson")
dem = rioxarray.open_rasterio(local_path / "las" / "combined.tif")
clipped = dem.rio.clip(polygon.geometry)
clipped.rio.to_raster(local_path / "las" / "combined_clipped.tif")

In [ ]:
zo = rioxarray.open_rasterio(local_path / "zo" / "combined.tif")
clipped = zo.rio.clip(polygon.geometry)
clipped.rio.to_raster(local_path / "zo" / "combined_clipped.tif")

## Add a background zo layer to the lower resolution DEM

In [5]:
zo_land = 0.2
zo_ocean = 0.004

zo = rioxarray.open_rasterio(local_path / "5m_dem_espiritu.nc").squeeze("band", drop=True)
zo.data[numpy.logical_not(numpy.isnan(zo.data))] = zo_land
zo.data[numpy.isnan(zo.data)] = zo_ocean

zo_out = xarray.DataArray(
    data=zo.data,
    dims=["y", "x"],
    coords=dict(x=(["x"], zo.x.data),
                y=(["y"], zo.y.data)),
    attrs=dict(
        description="Roughness length"
    ),
)
zo_out.rio.write_crs(zo.rio.crs, inplace=True)
zo.to_netcdf(local_path / "5m_zo_espiritu.nc", format="NETCDF4", engine="netcdf4")

## Load in 5m DEM and give -10m background

In [32]:
z_ocean = -10

z = rioxarray.open_rasterio(local_path / "5m_dem_espiritu.nc").squeeze("band", drop=True)
z.data[numpy.isnan(z.data)] = z_ocean

z_out = xarray.DataArray(
    data=z.data,
    dims=["y", "x"],
    coords=dict(x=(["x"], z.x.data),
                y=(["y"], z.y.data)),
    attrs=dict(
        description="Elevation"
    ),
)
z_out.rio.write_crs(z.rio.crs, inplace=True)
z_out.to_netcdf(local_path / "5m_z_espiritu_ocean.nc", format="NETCDF4", engine="netcdf4")

## Combine the 1m + 2m roughness maps into one grid
Clip to ROI from Alice - note did on NeSI and saved out the roughnes length at 5m as I was getting memory errors at 1m.

In [4]:
zo_5m = rioxarray.open_rasterio(local_path / "5m_zo_espiritu.nc").squeeze("band", drop=True)
zo_1m = rioxarray.open_rasterio(local_path / "zo" / "combined.tif").squeeze("band", drop=True)

In [5]:
region_of_interest = geopandas.read_file(local_path / "Domain_rect2.geojson")

In [6]:
zo_5m = zo_5m.rio.clip(region_of_interest.geometry, drop=True)

In [7]:
zo_5m.data = zo_5m.data.astype(numpy.float32)
zo_1m.data = zo_1m.data.astype(numpy.float32)

In [9]:
zo_5m.to_netcdf(local_path / "5m_zo_float32.nc", format="NETCDF4", engine="netcdf4")
zo_1m.to_netcdf(local_path / "1m_zo_float_32.nc", format="NETCDF4", engine="netcdf4")

In [6]:
zo_5m = rioxarray.open_rasterio(local_path / "5m_zo_float32.nc", chunks=True).squeeze("band", drop=True)
zo_1m = rioxarray.open_rasterio(local_path / "1m_zo_float_32.nc", chunks=True).squeeze("band", drop=True)

In [7]:
zo_1m_both = rioxarray.merge.merge_arrays([zo_5m, zo_1m], method="last")

In [14]:
numpy.arange(zo_1m.x.data[0], zo_1m.x.data[-1] + 5, 5)

array([729000., 729005., 729010., ..., 742990., 742995., 743000.])

In [13]:
numpy.arange(zo_1m.y.data[0], zo_1m.y.data[-1] - 5, -5)

array([8291675., 8291670., 8291665., ..., 8278010., 8278005., 8278000.])

In [15]:
zo_1m_to_5m=zo_1m.interp(x=numpy.arange(zo_1m.x.data[0], zo_1m.x.data[-1] + 5, 5), 
                         y=numpy.arange(zo_1m.y.data[0], zo_1m.y.data[-1] - 5, -5))

In [16]:
zo_1m_to_5m

<xarray.DataArray '__xarray_dataarray_variable__' (y: 2736, x: 2801)>
dask.array<transpose, shape=(2736, 2801), dtype=float32, chunksize=(2736, 2801), chunktype=numpy.ndarray>
Coordinates:
    spatial_ref  int32 0
  * x            (x) float64 7.29e+05 7.29e+05 7.29e+05 ... 7.43e+05 7.43e+05
  * y            (y) float64 8.292e+06 8.292e+06 ... 8.278e+06 8.278e+06
Attributes:
    add_offset:                0.0
    AREA_OR_POINT:             Area
    long_name:                 faceraster
    scale_factor:              1.0
    STATISTICS_APPROXIMATE:    YES
    STATISTICS_MAXIMUM:        0.13866476459095
    STATISTICS_MEAN:           0.021549481085802
    STATISTICS_MINIMUM:        0
    STATISTICS_STDDEV:         0.021044608594092
    STATISTICS_VALID_PERCENT:  68.62
    _FillValue:                -9999.0

In [17]:
zo_5m_both = rioxarray.merge.merge_arrays([zo_1m_to_5m, zo_5m], method="first")

In [19]:
zo_5m_both.to_netcdf(local_path / "5m_zo_combined.nc", format="NETCDF4", engine="netcdf4")